In [1]:
cd ../../

/Users/tpa/Code/snapml-examples/examples


In [2]:
from datasets import *

In [3]:
from sklearn.linear_model import LogisticRegression
from snapml import LogisticRegression as SnapLogisticRegression
import time
from sklearn.metrics import roc_auc_score
import pandas as pd

In [7]:
nrows = 100000

df = pd.DataFrame(columns=['n_samples', 'n_features', 't_fit_sklearn', 'score_sklearn', 
                           't_fit_snapml', 'score_snapml', 'speed_up', 'score_diff'])

for Dataset in [Higgs, Avazu]:
    
    dataset = Dataset('cache-dir')
    
    res = pd.Series(name=dataset.name, dtype='float64')
    
    X_train, X_test, y_train, y_test = dataset.get_train_test_split()
    X_train = X_train[:nrows, :]
    y_train = y_train[:nrows]
    
    res['n_samples'] = X_train.shape[0]
    res['n_features'] = X_train.shape[1]
    
    # scikit-learn
    lr = LogisticRegression(fit_intercept=False, n_jobs=4, class_weight='balanced')
    t0 = time.time()
    lr.fit(X_train, y_train)
    res['t_fit_sklearn'] = time.time()-t0
    res['score_sklearn'] = roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])
    
    # snapml
    lr = SnapLogisticRegression(fit_intercept=False, n_jobs=4, class_weight='balanced')
    t0 = time.time()
    lr.fit(X_train, y_train)
    res['t_fit_snapml'] = time.time()-t0
    res['score_snapml'] = roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])
    res['speed_up'] = res['t_fit_sklearn']/res['t_fit_snapml']
    res['score_diff'] = (res['score_snapml']-res['score_sklearn'])/res['score_sklearn']
    
    df = df.append(res)
    print(df)


       n_samples  n_features  t_fit_sklearn  score_sklearn  t_fit_snapml  \
Higgs   100000.0        28.0       2.043509       0.666532      0.068055   

       score_snapml   speed_up  score_diff  
Higgs       0.66682  30.027249    0.000432  
       n_samples  n_features  t_fit_sklearn  score_sklearn  t_fit_snapml  \
Higgs   100000.0        28.0       2.043509       0.666532      0.068055   
Avazu   100000.0   1000000.0      11.324797       0.706861      0.245819   

       score_snapml   speed_up  score_diff  
Higgs      0.666820  30.027249    0.000432  
Avazu      0.709019  46.069639    0.003054  
